## Deliverable 2. Create a Customer Travel Destinations Map.

In [56]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [57]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database /WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather Description
0,0,Cidreira,BR,-30.1811,-50.2056,73.83,81,67,18.45,broken clouds
1,1,Mosina,PL,52.2454,16.8471,31.33,86,0,4.61,clear sky
2,2,Misasi,TZ,-3.0000,33.0833,62.96,96,100,4.43,overcast clouds
3,3,Caravelas,BR,-17.7125,-39.2481,76.50,83,100,3.60,overcast clouds
4,4,Havelock,US,34.8791,-76.9013,58.95,72,100,5.75,overcast clouds


In [58]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [59]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather Description
2,2,Misasi,TZ,-3.0000,33.0833,62.96,96,100,4.43,overcast clouds
5,5,Kondagaon,IN,19.6000,81.6667,64.87,25,33,2.46,scattered clouds
9,9,Chuy,UY,-33.6971,-53.4616,66.56,92,100,11.48,overcast clouds
13,13,Xuddur,SO,4.1213,43.8894,65.82,77,95,10.42,overcast clouds
16,16,Mar Del Plata,AR,-38.0023,-57.5575,62.87,73,20,7.00,few clouds
17,17,Ribeira Grande,PT,38.5167,-28.7000,61.65,95,100,14.00,overcast clouds
19,19,Chicama,PE,-7.8447,-79.1469,68.54,71,86,5.32,overcast clouds
26,26,Quarai,BR,-30.3875,-56.4514,68.49,72,99,12.44,overcast clouds
27,27,Hermanus,ZA,-34.4187,19.2345,67.80,80,65,3.91,broken clouds
38,38,Adre,TD,13.4667,22.2000,63.41,16,14,12.55,few clouds


In [60]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0                     136
City                           136
Country                        132
Lat                            136
Lng                            136
Max Temp                       136
Humidity                       136
Cloudiness                     136
Wind Speed                     136
Current Weather Description    136
dtype: int64

In [61]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(axis=1, how='all', inplace=True)

/Users/Puneeth/opt/anaconda3/envs/PythonData/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [62]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather Description,Lat,Lng,Hotel Name
2,Misasi,TZ,62.96,overcast clouds,-3.0000,33.0833,
5,Kondagaon,IN,64.87,scattered clouds,19.6000,81.6667,
9,Chuy,UY,66.56,overcast clouds,-33.6971,-53.4616,
13,Xuddur,SO,65.82,overcast clouds,4.1213,43.8894,
16,Mar Del Plata,AR,62.87,few clouds,-38.0023,-57.5575,
17,Ribeira Grande,PT,61.65,overcast clouds,38.5167,-28.7000,
19,Chicama,PE,68.54,overcast clouds,-7.8447,-79.1469,
26,Quarai,BR,68.49,overcast clouds,-30.3875,-56.4514,
27,Hermanus,ZA,67.80,broken clouds,-34.4187,19.2345,
38,Adre,TD,63.41,few clouds,13.4667,22.2000,


In [63]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [64]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df=hotel_df.copy()

In [65]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
from os.path import exists
file_exists = exists(output_data_file)
if not file_exists:
    hotel_df.to_csv(output_data_file)
else:
    hotel_df_f = pd.read_csv(output_data_file)
    hotel_df_f.head()
    
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [66]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [67]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))